In [3]:
import pandas as pd
from pathlib import Path

import sys
sys.path.append('/Users/luis/Desktop/ETH/Courses/SS25-DSL/src')

from importlib import reload
from semantic_search import utils
reload(utils)

# Set pandas display options to show wider dataframes
pd.set_option('display.max_colwidth', None)  # Show full text in columns
pd.set_option('display.width', 1000)         # Set the display width
pd.set_option('display.max_columns', 20)     # Show more columns

In [57]:
from semantic_search.utils import collect_orig_paper_metadata, collect_ref_metadata

raw_dir = '/Users/luis/Desktop/ETH/Courses/SS25-DSL/raw-data/Conversions/opencvf-data/txt'
output_fpath = '/Users/luis/Desktop/ETH/Courses/SS25-DSL/raw-data/metadata/orig.csv'
collect_orig_paper_metadata(raw_dir, output_fpath, max_papers=5)

orig_metadata_fpath = '/Users/luis/Desktop/ETH/Courses/SS25-DSL/raw-data/metadata/orig.csv'
output_fpath = '/Users/luis/Desktop/ETH/Courses/SS25-DSL/raw-data/metadata/refs.csv'
collect_ref_metadata(orig_metadata_fpath, output_fpath)

Pulling OpenAlex metadata:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00,  3.00it/s]


100%|██████████| 1/1 [00:00<00:00,  1.99it/s]

## Misc

### Check papers with missing abstract

In [5]:
import re

raw_dir = '/Users/luis/Desktop/ETH/Courses/SS25-DSL/raw-data/Conversions/opencvf-data/txt'

def extract_abstract(fpath: str):
    doc_text = fpath.read_text(encoding="utf-8")
    abstract_match = re.search(r'## Abstract\n\n(.*?)(?=\n\n## \d+\.)', doc_text, re.DOTALL)
    return abstract_match.group(1) if abstract_match else ''

for fpath in Path(raw_dir).glob("*.txt"):
    if extract_abstract(fpath) == '':
        print(fpath)

# abstracts =[check_single_doc(fpath) for fpath in Path(raw_dir).glob("*.txt")]

/Users/luis/Desktop/ETH/Courses/SS25-DSL/raw-data/Conversions/opencvf-data/txt/Korycki_Class-Incremental_Mixture_of_Gaussians_for_Deep_Continual_Learning_CVPRW_2024_paper.txt
/Users/luis/Desktop/ETH/Courses/SS25-DSL/raw-data/Conversions/opencvf-data/txt/Hoque_IrrNet_Spatio-Temporal_Segmentation_Guided_Classification_for_Irrigation_Mapping_CVPRW_2024_paper.txt
/Users/luis/Desktop/ETH/Courses/SS25-DSL/raw-data/Conversions/opencvf-data/txt/Zhang_MOHO_Learning_Single-view_Hand-held_Object_Reconstruction_with_Multi-view_Occlusion-Aware_Supervision_CVPR_2024_paper.txt
/Users/luis/Desktop/ETH/Courses/SS25-DSL/raw-data/Conversions/opencvf-data/txt/Paissan_Structured_Sparse_Back-propagation_for_Lightweight_On-Device_Continual_Learning_on_Microcontroller_CVPRW_2024_paper.txt
/Users/luis/Desktop/ETH/Courses/SS25-DSL/raw-data/Conversions/opencvf-data/txt/Halawa_Multi-Task_Multi-Modal_Self-Supervised_Learning_for_Facial_Expression_Recognition_CVPRW_2024_paper.txt
/Users/luis/Desktop/ETH/Courses/SS2

## Testing crossref API

In [2]:
import pandas as pd

df = pd.read_csv('/Users/luis/Desktop/ETH/Courses/SS25-DSL/raw-data/metadata/openalex-ids+refs.csv')
df.head()

,fpath,title,doi,oaid,referenced_works
0,/cluster/home/lcarretero/workspace/dsl/dsl-res...,Fixed Point Diffusion Models,https://doi.org/10.1063/1.2121687,https://openalex.org/W2000456051,"['https://openalex.org/W1504980292', 'https://..."
1,/cluster/home/lcarretero/workspace/dsl/dsl-res...,BEVNeXt: Reviving Dense BEV Frameworks for 3D ...,https://doi.org/10.1109/cvpr52733.2024.01901,https://openalex.org/W4402727763,"['https://openalex.org/W1861492603', 'https://..."
2,/cluster/home/lcarretero/workspace/dsl/dsl-res...,Leveraging Pre-trained Multi-task Deep Models ...,https://doi.org/10.1109/cvprw63382.2024.00473,https://openalex.org/W4402916217,[]
3,/cluster/home/lcarretero/workspace/dsl/dsl-res...,Insights from the Use of Previously Unseen Neu...,https://doi.org/10.48550/arxiv.2404.02189,https://openalex.org/W4393967825,[]
4,/cluster/home/lcarretero/workspace/dsl/dsl-res...,Efficient local correlation volume for unsuper...,https://doi.org/10.1109/cvprw63382.2024.00049,https://openalex.org/W4402904316,"['https://openalex.org/W1513100184', 'https://..."


In [ ]:
ref_doi = '10.1109/TASC.2010.2088091'
crossref_url = f'https://api.crossref.org/works/{ref_doi}'
response = requests.get(crossref_url)
data = response.json()
data['message']['abstract']


{'indexed': {'date-parts': [[2025, 2, 21]],
  'date-time': '2025-02-21T10:19:07Z',
  'timestamp': 1740133147234,
  'version': '3.37.3'},
 'reference-count': 8,
 'publisher': 'Institute of Electrical and Electronics Engineers (IEEE)',
 'issue': '3',
 'license': [{'start': {'date-parts': [[2011, 6, 1]],
    'date-time': '2011-06-01T00:00:00Z',
    'timestamp': 1306886400000},
   'content-version': 'vor',
   'delay-in-days': 0,
   'URL': 'https://ieeexplore.ieee.org/Xplorehelp/downloads/license-information/IEEE.html'}],
 'funder': [{'DOI': '10.13039/100000015',
   'name': 'U.S. Department of Energy',
   'doi-asserted-by': 'publisher',
   'award': ['DEFG0210ER41650'],
   'id': [{'id': '10.13039/100000015',
     'id-type': 'DOI',
     'asserted-by': 'publisher'}]}],
 'content-domain': {'domain': [], 'crossmark-restriction': False},
 'short-container-title': ['IEEE Trans. Appl. Supercond.'],
 'published-print': {'date-parts': [[2011, 6]]},
 'abstract': "<jats:p>A 1.3 GHz test cavity has been

In [18]:
name_of_paper = df.iloc[1].title
crossref_url = f'https://api.crossref.org/works?query.bibliographic={name_of_paper}&rows=1'
response = requests.get(crossref_url)
data = response.json()
data.get('message', {}).get('items', [None])[0]
data

{'status': 'ok',
 'message-type': 'work-list',
 'message-version': '1.0.0',
 'message': {'facets': {},
  'total-results': 2004128,
  'items': [{'indexed': {'date-parts': [[2025, 4, 16]],
     'date-time': '2025-04-16T08:30:45Z',
     'timestamp': 1744792245500,
     'version': '3.28.0'},
    'reference-count': 73,
    'publisher': 'IEEE',
    'license': [{'start': {'date-parts': [[2024, 6, 16]],
       'date-time': '2024-06-16T00:00:00Z',
       'timestamp': 1718496000000},
      'content-version': 'stm-asf',
      'delay-in-days': 0,
      'URL': 'https://doi.org/10.15223/policy-029'},
     {'start': {'date-parts': [[2024, 6, 16]],
       'date-time': '2024-06-16T00:00:00Z',
       'timestamp': 1718496000000},
      'content-version': 'stm-asf',
      'delay-in-days': 0,
      'URL': 'https://doi.org/10.15223/policy-037'}],
    'content-domain': {'domain': [], 'crossmark-restriction': False},
    'published-print': {'date-parts': [[2024, 6, 16]]},
    'DOI': '10.1109/cvpr52733.2024.01

In [35]:
ref_df = pd.read_csv('/Users/luis/Desktop/ETH/Courses/SS25-DSL/raw-data/metadata/openalex-refs-abstracts.csv')
ref_df.head(1)

,oaid,title,abstract,type,topic,domain,field,subfield
0,https://openalex.org/W2194775991,Deep Residual Learning for Image Recognition,"Deeper neural networks are more difficult to train. We present a residual learning framework to ease the training of networks that are substantially deeper than those used previously. We explicitly reformulate the layers as learning residual functions with reference to the layer inputs, instead of learning unreferenced functions. We provide comprehensive empirical evidence showing that these residual networks are easier to optimize, and can gain accuracy from considerably increased depth. On the ImageNet dataset we evaluate residual nets with a depth of up to 152 layers - 8× deeper than VGG nets [40] but still having lower complexity. An ensemble of these residual nets achieves 3.57% error on the ImageNet test set. This result won the 1st place on the ILSVRC 2015 classification task. We also present analysis on CIFAR-10 with 100 and 1000 layers. The depth of representations is of central importance for many visual recognition tasks. Solely due to our extremely deep representations, we obtain a 28% relative improvement on the COCO object detection dataset. Deep residual nets are foundations of our submissions to ILSVRC & COCO 2015 competitions1, where we also won the 1st places on the tasks of ImageNet detection, ImageNet localization, COCO detection, and COCO segmentation.",article,Advanced Neural Network Applications,Physical Sciences,Computer Science,Computer Vision and Pattern Recognition


In [36]:
import requests
from tqdm import trange

fields_of_interest = ','.join(['DOI', 'title', 'type','references-count', 'reference', 'abstract'])

all_raw = []
for i in trange(200):
    name_of_paper = ref_df.iloc[i].title if i > 0 else 'Ultra-Gradient Test Cavity for Testing SRF Wafer Samples'
    crossref_url = f'https://api.crossref.org/works?query.bibliographic={name_of_paper}&rows=2&select={fields_of_interest}'
    response = requests.get(crossref_url)
    data = response.json()
    raw = data.get('message', {}).get('items', [None])[0]  # FIXME: Do proper relevance comparison as advised
    all_raw.append(raw)

  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [01:37<00:00,  2.05it/s]


In [37]:
ref_cnts = pd.Series([raw.get('references-count', 0) for raw in all_raw])
ref_cnts.values

array([  8,   0,  29,   0,  30,  14,   0,   0,  24,  10,   0,  91,  92,
        68,  23,  92,  28,  66, 100,  36,   0,   0,   0,  52,  97,   0,
         0,  33, 236,  24,   0,  86,  24,  55,  41,  39,  70,  31,  35,
        41, 125,  87,  13,  26,   0,  17,  58,  43,  73,  67,  62,  51,
        27,   0,  62,  48,  35,  28,  89,  57,  71,  81,  83,   0, 152,
        53,   0, 100, 136,  61,   0,   0, 121,  42, 117,  16,  55,  65,
        54,  22,  37,  76,   8,  28,  24,  56,  50,  25,   0,   0,   0,
        16,  26,  21, 106,  38,  29,  39,   0,  58,   0,  76,   0,   0,
        63,  48,   0,   0,  38,  33,  18,  44,   9,  60,   0,  53,  19,
         0,  47,  95,   0,  39,   0,   0,  64,   0,  21,  60,  79,   1,
         0, 287,   0,   0,  55, 168,  12,   0,  33,   0,   0,  61,  53,
        54,  36,  38,  62,  43,  56,  61,  30,  81,  60,  62,   0,  40,
        47,  21, 210,   0,   0,  43,  36,  51,  44,  48,   0,  60,  24,
         0,  47,  77,  56,  57,  19,  45,  48,  27,  59,  64,  3

In [39]:
all_raw[3]

{'DOI': '10.21123/bsj.2024.11089',
 'title': ['Improved Deep Perceptual Hashing Algorithm (IDP-HA), Information Retrieval System, Microsoft Common Objects In Context (MS COCO), Remote Cloud Computing, Computer science, information systems',
  'نظام محسّن لاسترجاع المعلومات قائم على خوارزمية التجزئة الإدراكية العميقة للحوسبة السحابية عن بعد'],
 'type': 'journal-article',
 'references-count': 0,
 'abstract': '<jats:p>يمكن أن يعزى نمو استرجاع المعلومات والخدمات المرتبطة بها إلى التقدم التقني. وفي الوقت نفسه، تتأثر الطرق التقليدية لاسترجاع المعلومات بقيود الأداء والدقة وقابلية التوسع. يعد نظام استرجاع المعلومات للحوسبة السحابية البعيدة الذي يعتمد على خوارزمية التجزئة الإدراكية العميقة المحسنة (IDP-HA) أحد الحلول التي تم تطويرها لحل هذه القيود. تُستخدم الأنظمة على نطاق واسع نظرًا لقدرتها على التعرف على الأنماط المعقدة في البيانات. لا تزال دقة قياس تشابه المعلومات غير متوفرة بسبب التعقيد الكامن في البيانات وطرق القياس. يستخدم أسلوب التجزئة الإدراكي العميق أطر عمل الشبكة العصبية العميقة (DNN)

In [38]:
abstracts = pd.Series([raw.get('abstract', '') for raw in all_raw])
abstracts.values

array(["<jats:p>A 1.3 GHz test cavity has been designed to test wafer samples of superconducting materials. This mushroom shaped cavity, operating in TE<jats:sub>01</jats:sub>mode, creates a unique distribution of surface fields. The surface magnetic field on the sample wafer is 3.75 times greater than elsewhere on the Niobium cavity surface. This field design is made possible through dielectrically loading the cavity by locating a hemisphere of ultra-pure sapphire just above the sample wafer. The sapphire pulls the fields away from the walls so the maximum field the Nb surface sees is 25% of the surface field on the sample. In this manner, it should be possible to drive the sample wafer well beyond the BCS limit for Niobium while still maintaining a respectable Q. The sapphire's purity must be tested for its loss tangent and dielectric constant to finalize the design of the mushroom test cavity. A sapphire loaded CEBAF cavity has been constructed and tested. The results on the dielect